In [ ]:
import ipywidgets as widgets


TILED HELPER FUNCTIONS

In [ ]:
from tiled.client import from_uri

In [ ]:
# Gets the metadata entry in the dropdown for each scan with the tiled_uri for that scan
# In segmentation case, it gets job_name value and segmentation uri as (key,value) pairs
def get_tiled_uri_metadata_field_entries(base_uri, uri_additional, metadata_field):
    """
    base_uri - str, tiled base uri (ex: looks like http://127.0.0.1:8888)
    uri_additional - str, the add on path to the base_uri (ex: could be something like "node_name_1" or "" 
                    based on your file structure)
    metadata_field - str, the field to extract
    """
    try:
        full_tiled_uri = base_uri+"/api/v1/metadata/"+uri_additional
        client = from_uri(full_tiled_uri)
        dropdown_list = []
        

        for experiment in client:
            child_node_client = client[experiment]["seg_result"]
            scan_uri = child_node_client.uri
            metadata = child_node_client.metadata
            metadata_field_entry = metadata[metadata_field]
            dropdown_list.append((metadata_field_entry,scan_uri))
            
        return dropdown_list

    except Exception as e:
        error_message = f"Error retrieving experiment names: {e}"
        print(error_message)
        return []

In [ ]:
def tiled_widget_refresh_callback(dropdown_widget):
    new_list_experiments = get_tiled_uri_metadata_field_entries("http://127.0.0.1:8888","","job_name")
    dropdown_widget.options = new_list_experiments
    dropdown_widget.value = None

In [ ]:
scan_uri_metadata_tuples = get_tiled_uri_metadata_field_entries("http://127.0.0.1:8888","","job_name")

tiled_file_widget = widgets.Dropdown(
    options=scan_uri_metadata_tuples,
    value=None,
    description='Experiment name:',
)

tiled_widget_refresh_button = widgets.Button(description="Refresh")

display(tiled_file_widget, tiled_widget_refresh_button)

def tiled_refresh_button_click(button):
    tiled_widget_refresh_callback(tiled_file_widget)

tiled_widget_refresh_button.on_click(tiled_refresh_button_click)


In [ ]:
tiled_file_widget.value

Plot the segmentation result

In [ ]:
# With the selected job_name in the dropdown, you get the scan_uri
# Get the mask uri and plot the classes

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

In [ ]:

def plot_seg_mask(selected_scan_uri, slice_idx, axs):
    scan_client = from_uri(selected_scan_uri)
    scan_client_metadata = scan_client.metadata
    mask_uri = scan_client_metadata["mask_uri"]
    results_uri = mask_uri[:-5]

    mask = scan_client["mask"][slice_idx]
    mask_idx = scan_client_metadata["mask_idx"]

    if slice_idx > len(mask_idx):
        raise ValueError("Slice index out of range")

    # Extract class information 
    result_client = from_uri(results_uri)
    result_metadata = result_client.metadata
    class_meta_data = result_metadata["classes"]
    max_class_id = len(class_meta_data.keys()) - 1

    colors = [
        annotation_class["color"] for _, annotation_class in class_meta_data.items()
    ]
    labels = [
        annotation_class["label"] for _, annotation_class in class_meta_data.items()
    ]
    # Add color for unlabeled pixels
    colors = ["#D3D3D3"] + colors
    labels = ["Unlabeled"] + labels

    axs.imshow(
        mask,
        cmap=ListedColormap(colors),
        vmin=-1.5,
        vmax=max_class_id + 0.5,
    )
    axs.set_title(result_metadata["project_name"] + ", slice: " + mask_idx[slice_idx])

    # create a patch for every color
    patches = [
        mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(labels))
    ]
    # Plot legend below the image
    axs.legend(
        handles=patches, loc="upper center", bbox_to_anchor=(0.5, -0.075), ncol=3
    )

In [ ]:
selected_scan_uri = tiled_file_widget.value

fig = plt.figure(layout="constrained")
ax_array = fig.subplots(2, 2, squeeze=False)

plot_seg_mask(selected_scan_uri,0,[0,0])
plot_seg_mask(selected_scan_uri,1,[0,1])
plot_seg_mask(selected_scan_uri,2,[1,0])
plot_seg_mask(selected_scan_uri,3,[1,1])
